In [1]:
# ===== PyTorch (CUDA 12.1) =====
!pip install -q torch==2.1.0+cu121 torchvision==0.16.0+cu121 torchaudio==2.1.0+cu121 \
  --index-url https://download.pytorch.org/whl/cu121

# ===== PyTorch Geometric (prebuilt wheels, no compilation) =====
!pip install -q torch-geometric

# ===== Fairness + utilities =====
!pip install -q fairlearn scikit-learn pandas numpy



ERROR: Could not find a version that satisfies the requirement torch==2.1.0+cu121 (from versions: 2.2.0+cu121, 2.2.1+cu121, 2.2.2+cu121, 2.3.0+cu121, 2.3.1+cu121, 2.4.0+cu121, 2.4.1+cu121, 2.5.0+cu121, 2.5.1+cu121)
ERROR: No matching distribution found for torch==2.1.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 13.6 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [3]:
!git clone https://github.com/ananyabatra04/fairness-graph-gnn.git

Cloning into 'fairness-graph-gnn'...
remote: Enumerating objects: 22, done.
remote: Total 22 (delta 0), reused 0 (delta 0), pack-reused 22 (from 1)
Receiving objects: 100% (22/22), 52.28 MiB | 10.71 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (9/9), done.


In [4]:
nodes = pd.read_csv("fairness-graph-gnn/datasets/region_job.csv")
feature_cols = [
    c for c in nodes.columns
    if c not in ["user_id", "region", "I_am_working_in_field"]
]
nodes["I_am_working_in_field"].value_counts().sort_index()

# Drop unknown labels
nodes = nodes[nodes["I_am_working_in_field"] != -1]

# Binarize: 0 -> 0, 1–4 -> 1
nodes["label"] = (nodes["I_am_working_in_field"] > 0).astype(int)

nodes["label"].value_counts()
nodes["label"].unique()


array([1, 0])

In [5]:
import torch

X = torch.tensor(
    nodes[feature_cols].values,
    dtype=torch.float
)

X = (X - X.mean(dim=0)) / (X.std(dim=0) + 1e-6) #normalization


y = torch.tensor(
    nodes["label"].values,
    dtype=torch.long
)

sensitive = torch.tensor(
    (nodes["region"] == "Bratislava").astype(int).values,
    dtype=torch.long
)
X.shape, y.shape, sensitive.shape


(torch.Size([10262, 276]), torch.Size([10262]), torch.Size([10262]))

In [6]:
scaler = StandardScaler()
X = torch.tensor(
    scaler.fit_transform(X),
    dtype=torch.float
)

In [7]:
edges = pd.read_csv(
    "fairness-graph-gnn/datasets/region_job_relationship.txt",
    sep="\t",
    names=["src", "dst"]
)



In [8]:
#GCNs require node indices from 0 … N-1. Pokec user IDs are arbitrary.

id_map = {
    uid: i for i, uid in enumerate(nodes["user_id"].values)
}

src_list = []
dst_list = []

for src, dst in zip(edges["src"], edges["dst"]):
    if src in id_map and dst in id_map:
        src_list.append(id_map[src])
        dst_list.append(id_map[dst])

edge_index = torch.tensor(
    [src_list, dst_list],
    dtype=torch.long
)

# make undirected
edge_index = torch.cat(
    [edge_index, edge_index.flip(0)],
    dim=1
)


In [9]:
edge_index.min(), edge_index.max(), X.size(0)


(tensor(1), tensor(10260), 10262)

In [10]:
num_nodes = nodes.shape[0]

perm = torch.randperm(num_nodes)

train_size = int(0.6 * num_nodes)
val_size   = int(0.2 * num_nodes)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask   = torch.zeros(num_nodes, dtype=torch.bool)
test_mask  = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[perm[:train_size]] = True
val_mask[perm[train_size:train_size+val_size]] = True
test_mask[perm[train_size+val_size:]] = True


In [11]:
from torch_geometric.data import Data

data = Data(
    x=X,
    edge_index=edge_index,
    y=y
)

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask
data.sensitive = sensitive


In [12]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


In [13]:
device = torch.device("cpu")
model = GCN(
    in_channels=data.num_features,
    hidden_channels=64,
    out_channels=2
).to(device)

data = data.to(device)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.01,
    weight_decay=5e-4
)

criterion = torch.nn.CrossEntropyLoss()


In [14]:
def train():
    model.train()
    optimizer.zero_grad()

    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])

    loss.backward()
    optimizer.step()

    return loss.item()

@torch.no_grad()
def test(mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    correct = (pred[mask] == data.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc



In [18]:
@torch.no_grad()
def fairness_metrics(y_true, y_pred, sensitive):
    """
    y_true: (N,) ground truth labels {0,1}
    y_pred: (N,) predicted labels {0,1}
    sensitive: (N,) sensitive attribute {0,1}
    """

    metrics = {}

    for s_val in [0, 1]:
        mask_s = (sensitive == s_val)

        # P(ŷ = 1 | s)
        if mask_s.sum() > 0:
            metrics[f"P_yhat1_s{s_val}"] = y_pred[mask_s].float().mean()
        else:
            metrics[f"P_yhat1_s{s_val}"] = torch.tensor(0.0)

        # P(ŷ = 1 | y = 1, s)
        mask_y1_s = mask_s & (y_true == 1)
        if mask_y1_s.sum() > 0:
            metrics[f"TPR_s{s_val}"] = y_pred[mask_y1_s].float().mean()
        else:
            metrics[f"TPR_s{s_val}"] = torch.tensor(0.0)

    # Fairness gaps
    delta_sp = torch.abs(
        metrics["P_yhat1_s0"] - metrics["P_yhat1_s1"]
    )

    delta_eo = torch.abs(
        metrics["TPR_s0"] - metrics["TPR_s1"]
    )

    return delta_sp.item(), delta_eo.item()

@torch.no_grad()
def evaluate(mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)

    acc = (pred[mask] == data.y[mask]).float().mean().item()

    delta_sp, delta_eo = fairness_metrics(
        y_true=data.y[mask],
        y_pred=pred[mask],
        sensitive=data.sensitive[mask]
    )

    return acc, delta_sp, delta_eo


In [15]:
device = torch.device("cpu")
data = data.to(device)
model = model.to(device)

In [19]:
for epoch in range(1, 201):
    loss = train()

    if epoch % 20 == 0:
        train_acc, _, _ = evaluate(data.train_mask)
        val_acc, val_sp, val_eo = evaluate(data.val_mask)
        test_acc, test_sp, test_eo = evaluate(data.test_mask)

        print(
            f"Epoch {epoch:03d} | "
            f"Loss {loss:.4f} | "
            f"Train {train_acc:.3f} | "
            f"Val {val_acc:.3f} | "
            f"Test {test_acc:.3f} | "
            f"ΔSP {test_sp:.3f} | "
            f"ΔEO {test_eo:.3f}"
        )


Epoch 020 | Loss 0.2804 | Train 0.914 | Val 0.684 | Test 0.691 | ΔSP 0.524 | ΔEO 0.695
Epoch 040 | Loss 0.2741 | Train 0.923 | Val 0.679 | Test 0.692 | ΔSP 0.505 | ΔEO 0.678
Epoch 060 | Loss 0.2677 | Train 0.923 | Val 0.672 | Test 0.681 | ΔSP 0.513 | ΔEO 0.676
Epoch 080 | Loss 0.2656 | Train 0.921 | Val 0.674 | Test 0.688 | ΔSP 0.495 | ΔEO 0.666
Epoch 100 | Loss 0.2621 | Train 0.916 | Val 0.676 | Test 0.686 | ΔSP 0.495 | ΔEO 0.665
Epoch 120 | Loss 0.2620 | Train 0.926 | Val 0.673 | Test 0.686 | ΔSP 0.519 | ΔEO 0.686
Epoch 140 | Loss 0.2586 | Train 0.931 | Val 0.677 | Test 0.684 | ΔSP 0.513 | ΔEO 0.679
Epoch 160 | Loss 0.2577 | Train 0.925 | Val 0.683 | Test 0.687 | ΔSP 0.517 | ΔEO 0.685
Epoch 180 | Loss 0.2551 | Train 0.934 | Val 0.667 | Test 0.688 | ΔSP 0.513 | ΔEO 0.683
Epoch 200 | Loss 0.2476 | Train 0.937 | Val 0.670 | Test 0.679 | ΔSP 0.517 | ΔEO 0.677
